<a href="https://colab.research.google.com/github/amilyk/transformers-for-NLP/blob/main/sequence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets transformers seqeval

In [ ]:
# !pip install -U evaluate

In [ ]:
task = "ner" #需要是"ner", "pos" 或者 "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_dataset

In [ ]:
# dataset = load_dataset("glue", "cola")

In [ ]:
# !pip install "datasets==2.19.0" "fsspec==2023.9.2"

In [ ]:
dataset = load_dataset("conll2003")

/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
dataset["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
label_list = dataset["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
show_random_elements(dataset["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,12116,"[-, Government, will, finish, pension, reform, before, the, year, 2000]","[8, 22, 20, 37, 21, 21, 15, 12, 21, 11]","[0, 11, 21, 22, 11, 12, 17, 11, 12, 12]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1823,"["", The, strategic, aim, of, European, integration, should, not, in, any, way, damage, Ukraine, 's, interests, in, post-Soviet, areas, .]","[0, 12, 16, 21, 15, 16, 21, 20, 30, 15, 12, 21, 21, 22, 27, 24, 15, 21, 24, 7]","[0, 11, 12, 12, 13, 11, 12, 21, 22, 13, 11, 12, 12, 12, 11, 12, 13, 11, 12, 0]","[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 7, 0, 0]"
2,11551,"[Sept, comi, medium-term, bond, up, three, at, 109.45, .]","[22, 24, 16, 21, 15, 11, 15, 11, 7]","[11, 12, 11, 12, 13, 11, 13, 11, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,894,"[division, match, on, Thursday, :]","[21, 21, 15, 22, 8]","[11, 12, 13, 11, 0]","[0, 0, 0, 0, 0]"
4,12102,"[PRESS, DIGEST, -, Spain, -, Aug, 28, .]","[24, 22, 8, 22, 8, 22, 11, 7]","[11, 12, 0, 11, 0, 11, 12, 0]","[0, 0, 0, 5, 0, 0, 0, 0]"
5,671,"[NEW, YORK, 1996-08-22]","[22, 22, 11]","[11, 12, 12]","[5, 6, 0]"
6,5233,"[Played, Friday, :]","[22, 22, 8]","[11, 12, 0]","[0, 0, 0]"
7,11139,"[The, CRP, said, the, four, were, arrested, on, Monday, night, at, the, northeastern, Tafawa, Balewa, University, .]","[12, 22, 38, 12, 11, 38, 40, 15, 22, 21, 15, 12, 16, 22, 22, 22, 7]","[11, 12, 21, 11, 12, 21, 22, 13, 11, 12, 13, 11, 12, 12, 12, 12, 0]","[0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0]"
8,8785,"[5., Wim, Omloop, (, Belgium, ), Collstrop, same, time]","[11, 22, 22, 4, 22, 5, 37, 16, 21]","[11, 12, 12, 0, 11, 0, 21, 11, 12]","[0, 1, 2, 0, 5, 0, 3, 0, 0]"
9,9859,"[June, mill, output, data, were, revised, slightly, downward, to, 210,622, tonnes, from, the, preliminary, 210,683, ,, while, shipments, were, revised, marginally, upward, to, 213,989, tonnes, from, 213,845, .]","[22, 21, 21, 24, 38, 40, 30, 30, 35, 11, 24, 15, 12, 16, 11, 6, 15, 24, 38, 40, 30, 30, 35, 11, 24, 15, 11, 7]","[11, 12, 12, 12, 21, 11, 3, 4, 13, 11, 12, 13, 11, 12, 11, 0, 17, 11, 21, 11, 1, 3, 13, 11, 12, 13, 11, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


# 预处理数据

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
example = dataset["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [ ]:
#多个子token对应一个word，还是多个子token的第一个对应word，其他子token设为none

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(dataset['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

# 微调

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    f"test-{task}",
    # evaluation_strategy = "epoch",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
# import transformers
# print(transformers.__version__)   # 例如 4.44.2、4.46.0 等


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# !pip install -U evaluate
# !pip install seqeval

In [ ]:
import evaluate          # 需要先 pip install evaluate
metric = evaluate.load("seqeval")

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
{'LOC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

{'LOC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3203160651.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xunkangzju (xunkangzju-zj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.242900,0.070840,0.898916,0.918224,0.908467,0.979665
2,0.051600,0.061900,0.921053,0.935787,0.928361,0.982763
3,0.030200,0.063192,0.926123,0.936794,0.931428,0.983526


TrainOutput(global_step=2634, training_loss=0.08508374280067856, metrics={'train_runtime': 449.0252, 'train_samples_per_second': 93.81, 'train_steps_per_second': 5.866, 'total_flos': 510122266253334.0, 'train_loss': 0.08508374280067856, 'epoch': 3.0})

In [ ]:
# api-key="d8d8c5d293f2ab1a0c1b3cebc85cbb4e86ffa4a3"

In [ ]:
trainer.evaluate()

{'eval_loss': 0.06319183856248856,
 'eval_precision': 0.9261225392612253,
 'eval_recall': 0.9367938248126189,
 'eval_f1': 0.9314276180412657,
 'eval_accuracy': 0.9835258233116749,
 'eval_runtime': 5.5154,
 'eval_samples_per_second': 589.254,
 'eval_steps_per_second': 36.987,
 'epoch': 3.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': np.float64(0.9539274924471299),
  'recall': np.float64(0.9648586707410237),
  'f1': np.float64(0.9593619445499431),
  'number': np.int64(2618)},
 'MISC': {'precision': np.float64(0.807936507936508),
  'recall': np.float64(0.8269699431356621),
  'f1': np.float64(0.8173424327579285),
  'number': np.int64(1231)},
 'ORG': {'precision': np.float64(0.8915950334288443),
  'recall': np.float64(0.9080739299610895),
  'f1': np.float64(0.8997590361445784),
  'number': np.int64(2056)},
 'PER': {'precision': np.float64(0.9746710526315789),
  'recall': np.float64(0.9765985497692815),
  'f1': np.float64(0.9756338491932829),
  'number': np.int64(3034)},
 'overall_precision': np.float64(0.9261225392612253),
 'overall_recall': np.float64(0.9367938248126189),
 'overall_f1': np.float64(0.9314276180412657),
 'overall_accuracy': 0.9835258233116749}